In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

import os
files_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files_paths.append(os.path.join(dirname, filename)) 

As  a bigginer data scientist, I tghout to do some EDA on sample datasets, Nothing more simple than the amazon best sellers books to start with<br>
With this dataset we re going to learn some critical thinking that data scientists must have, for this reason we re going to follow this timeline:
* Discover the data 
* Ask some questions 
* Make some plots to find out what the data is hiding 

## Import and discover the data

In [ ]:
data = pd.read_csv(files_paths[0])
display(data.head(5))

#### Data discption

In [ ]:
print("the dataset is made of {} examples and {} features".format(data.shape[0], data.shape[1]))

#### Data statistics

In [ ]:
unique_name = data['Name'].unique()
unique_author = data['Author'].unique()
years_range = (data['Year'].min(), data['Year'].max())
unique_genre = data['Genre'].unique()

print("The dataset contains {} unique book names".format(len(unique_name)))
print("The dataset contains {} unique authors".format(len(unique_author)))
print("The range year of the collected data is between {} and {}".format(years_range[0], years_range[1]))
print("The dataset contains {} unique genre".format(len(unique_genre)))

There is 351 unique book names out of 550, may be we have duplicated data ? <br>
let's discover this 

In [ ]:
dup = data.duplicated()

print("The number of duplicated rows in data is {} ".format(dup[dup].shape[0]))

Which means that we dont have any duplicated data to ride of, Yupi !!

## Questions 

In my opinion good questions are the right path to handle every data science project, especially during the first steps where we re starting to dicover the data

For that reason the question I will answer in thi snotebook are the folowing :
1. What kind of books are most sold ?
1. Is the price distributions of fiction and non fiction books are different ?
1. What is the book genre that is more reviewd and have best review rating ?
1. Finnaly, I added a demographic filtering, and I got inspired by this notebook https://www.kaggle.com/accountstatus/amazon-top-50-books-eda-and-filtering

## DATA Visualization

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(data.Year, hue=data.Genre)
plt.show()

Non fiction books are more byuyer than the fiction ones

### Price of fiction vs non fiction books

In [ ]:
import matplotlib.patches as mpatches

plt.hist(data[data.Genre == 'Non Fiction']['Price'])
plt.hist(data[data.Genre == 'Fiction']['Price'])
red_patch = mpatches.Patch(color='orange', label='The Price of non fictional books')
blue_patch = mpatches.Patch(color='blue', label='The Price of fictional books')
plt.legend(handles=[red_patch, blue_patch])
plt.show()

The price of non fictional books is more higher than the fictional books

### Comparing Mean price of fictional and non fictional Books

In [ ]:
fictional_mean_price = data[data.Genre == 'Fiction']['Price'].mean()
non_fictional_mean_price = data[data.Genre == 'Non Fiction']['Price'].mean()

print('The mean price of fictional books is %.2f$ and the mean price of non fictional books is %.2f$ with a difference of %.2f$' %(fictional_mean_price, non_fictional_mean_price, abs(fictional_mean_price-non_fictional_mean_price)))

### Rating of Fictional and non foctional books

In [ ]:
plt.hist(data[data.Genre == 'Non Fiction']['User Rating'])
plt.hist(data[data.Genre == 'Fiction']['User Rating'])
red_patch = mpatches.Patch(color='orange', label='The User Rating of non fictional books')
blue_patch = mpatches.Patch(color='blue', label='The User Rating of fictional books')
plt.legend(handles=[red_patch, blue_patch])
plt.show()

It is clear that the non fictional books get the best user rating, but the data is right skeewed. 
we can inerprete that only people that are satisfied with their purchases review their books, and for me it seems right because even for the fictional books the user rating is slightly good 

### Relation between User rating and Reviews

In [ ]:
sns.jointplot(x='User Rating', y='Reviews', data=data, kind='kde', color='r')

### Demographic Filtering

In [ ]:
c = data['User Rating'].mean()

In [ ]:
min_reviews = 5000
books = data[data['Reviews'] >= min_reviews]

In [ ]:
def weighted_rating(x, m=min_reviews, c=c):
    v = x['Reviews']
    r = x['User Rating']
    
    return (r*v/(v+m))+(c*m/(v+m))

In [ ]:
score = weighted_rating(books, m=min_reviews, c=c)

In [ ]:
books['Scores'] = data.apply(lambda x:weighted_rating(x, min_reviews, c), axis=1)

In [ ]:
books.sort_values('Scores',ascending=False, inplace=True)

In [ ]:
books.head(10)